In [1]:
import sys, os, yaml
import torch
from pprint import pprint
import torchvision
from torchmetrics.detection.map import MeanAveragePrecision
ROOT_DIR = None
with open('../cfg/project/default.yaml', 'r') as f: ROOT_DIR = yaml.load(f, Loader=yaml.FullLoader)['root_dir']
sys.path.insert(0, os.path.abspath(f"{ROOT_DIR}"))

from pl_jdt.data.datasets.penn_fundan import PennFudanDataset
from pl_jdt.utils.references.detection.engine import train_one_epoch, evaluate
import pl_jdt.utils.references.detection.utils as utils
import pl_jdt.utils.references.detection.transforms as T
from pl_jdt.utils.references.detection.coco_utils import get_coco_api_from_dataset
from pl_jdt.utils.references.detection.coco_eval import CocoEvaluator
import pl_jdt.utils.references.detection.utils as utils
# import pl_jdt.utils.transforms as T
from pl_jdt.models.mask_rcnn import get_mask_rcnn

NUM_THREADS = 4
DATA_PATH = f'{ROOT_DIR}/data/PennFudanPed'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
cpu_device = torch.device("cpu")

In [2]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

In [3]:
# preds = [
#     dict(
#         boxes=torch.Tensor([[258.0, 41.0, 606.0, 285.0]]),
#         scores=torch.Tensor([0.536]),
#         labels=torch.IntTensor([0]),
#     )
# ]
# target = [
#     dict(
#         boxes=torch.Tensor([[214.0, 41.0, 562.0, 285.0]]),
#         labels=torch.IntTensor([0]),
#     )
# ]
# metric = MeanAveragePrecision()
# metric.update(preds, target)
# pprint(metric.compute())

In [7]:
model_kwargs = {
    'num_classes': 2,
    'hidden_layer': 256
}
model = get_mask_rcnn(**model_kwargs)
model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [23]:
dataset = PennFudanDataset(DATA_PATH, get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
coco = get_coco_api_from_dataset(data_loader.dataset)
iou_types = _get_iou_types(model)
coco_evaluator = CocoEvaluator(coco, iou_types)
metric = MeanAveragePrecision()
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    for i, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)

        torch.cuda.synchronize()
        outputs = model(images)
        if i == 0:
            plt.imshow(images[0].cpu().permute(1, 2, 0).numpy())
            print("\ntargets",targets[0])
            print("\noutputs",outputs[0])

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        targets = [{k: v.to(cpu_device) for k, v in t.items()} for t in targets]

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        coco_evaluator.update(res)
        metric.update(outputs,targets)
        if i == 2: break
coco_evaluator.synchronize_between_processes()

# accumulate predictions from all images
coco_evaluator.accumulate()
coco_evaluator.summarize()
torch.set_num_threads(NUM_THREADS)
acc = metric.compute()
pprint(acc)

creating index...
index created!

targets {'boxes': tensor([[159., 181., 301., 430.],
        [419., 170., 534., 485.]]), 'labels': tensor([1, 1]), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8), 'image_id': tensor([0]), 'area': tensor([35358., 36225.]), 'iscrowd': tensor([0, 0])}

outputs {'boxes': tensor([[4.9289e+02, 1.4244e+02, 5.5900e+02, 2.2534e+02],
        [1.4593e+02, 1.6498e+02, 2.2273e+02, 2.4122e+02],
        [1.4760e+02, 2.3869e+02, 1.6334e+02, 2.5484e+02],
        [4.8859e+02, 1.4659e+02, 5.4913e+02, 3.2875e+02],
        [2.1023e-01, 3.0913e+02, 4

In [24]:
dataset = PennFudanDataset(DATA_PATH, get_transform(train=False))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
coco = get_coco_api_from_dataset(data_loader.dataset)
iou_types = _get_iou_types(model)
coco_evaluator = CocoEvaluator(coco, iou_types)
metric = MeanAveragePrecision()

model.eval()
with torch.no_grad():
    for i, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)

        torch.cuda.synchronize()
        outputs = model(images)
        if i == 0:
            plt.imshow(images[0].cpu().permute(1, 2, 0).numpy())
            print("\ntargets",targets[0])
            print("\noutputs",outputs[0])

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        targets = [{k: v.to(cpu_device) for k, v in t.items()} for t in targets]

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        coco_evaluator.update(res)
        metric.update(outputs,targets)
        if i == 2: break
coco_evaluator.synchronize_between_processes()

# accumulate predictions from all images
coco_evaluator.accumulate()
coco_evaluator.summarize()
torch.set_num_threads(NUM_THREADS)
acc = metric.compute()
pprint(acc)

creating index...
index created!

targets {'boxes': tensor([[159., 181., 301., 430.],
        [419., 170., 534., 485.]]), 'labels': tensor([1, 1]), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8), 'image_id': tensor([0]), 'area': tensor([35358., 36225.]), 'iscrowd': tensor([0, 0])}

outputs {'boxes': tensor([[4.9289e+02, 1.4244e+02, 5.5900e+02, 2.2534e+02],
        [1.4593e+02, 1.6498e+02, 2.2273e+02, 2.4122e+02],
        [1.4760e+02, 2.3869e+02, 1.6334e+02, 2.5484e+02],
        [4.8859e+02, 1.4659e+02, 5.4913e+02, 3.2875e+02],
        [2.1023e-01, 3.0913e+02, 4